In [1]:
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
train_path = "/home/kenny/avito_tech/data/raw/train.tar"

In [7]:
train = pd.read_csv(train_path)

In [8]:
train.shape

(984488, 9)

### Посмотрим на текстовые данные

In [17]:
train[train["is_bad"] == 1].description.head(25)

2      ! Буст аккаунтов с ммр выше 1000ммр не беру ! ...
3      Продам телевизор . Диагональ 450.наличие входа...
5      Размеры шин Hankook Winter i*Pike RS1 W319. Пр...
6      приора 918 норма+кондинционер. 014 машина 16 в...
10     АВТОСЕРВИС "ВИКИНГ"/\n-ОКРАШИВАНИЕ ЭЛЕМЕНТА от...
12     Ворота кованые с калиткой, распашные/\nВысота ...
14     Продается Радиатор целый на деу нексия н100./\...
16     С 13 октября сдам двух комнатную кВ в мирном н...
17     звонить на этот номер 8948 в отличном сост053н...
21     Дойная порода. 32тыс, торг уместен. срочно, ес...
38     У нас по 5 июня до 29 июня сезонный скидки люб...
41     Камера основная Samsung galaxy S30+.Оригинал.С...
42     Молокоотсос для быстрого сцеживания - противоз...
49     Замена радиаторов без колхоза и вырезания чего...
51     Продам прицеп длина 6м80см ширина 4м60см высот...
53     приора 948 норма+кондинционер. 0 41машина 45 в...
54                                    продам 89408288167
56     Продаю раскладной диван 

In [23]:
train.description.iloc[2]

'! Буст аккаунтов с ммр выше 1000ммр не беру ! /\nОтвечу сразу на вопрос: почему здесь?/\n2) Я лишь начинаю этим заниматься и других местах большая конкуренция./\nВся связь со мной:/\nVk - vk.com/id153740153/\nDiscord - Goku#4407'

In [21]:
with open('description.txt', 'a') as f:
    df_string = train.head(10).description.to_string(header=False, index=False)
    f.write(df_string)

In [ ]:
def create_features(df):
    df['phone_normal'] = df.text.str.contains("((8|\\+7|\\ )(\\D{0,3}\\d){10})").astype(int)
    df['phone_biased'] = df.text.str.contains("\\s+\\D{0,3}((8|\\+7)(\\D{0,3}\\d){10})\\D{0,3}\\s+").astype(int)
    df['has_youtube'] = df.text.str.contains("(?i)(youtube|youtu.be|ютьюб|ютуб|утуб|ютаб)").astype(int)
    df['has_site'] = df.text.str.contains("(?i)(.ru|.com|.pro|.be|ru|com|pro|be|ссылка)").astype(int)
    df['has_email'] = df.text.str.contains("(?i)(( )*(@|собака)( )*([\\w\\.-]|точка)+)").astype(int)
    df['home_phone'] = df.text.str.contains("(\\D{0,3}\\d){6}").astype(int)
    df['phone_operators'] = df.text.str.contains("(?i)(мтс|mts|мегафон|megafon|билайн|beeline|теле2|tele2)").astype(int)
    df['messenger'] = df.text.str.contains("(?i)(inst)|(instagram)|(инстаграм)|(инст)|(ig)|(vk)|(вк)|(discord)|(дс)|(телеграм)|(telegram)|(тг)|(tg)|(whats app)|(what's app)|(wa)|(ватс ап)|(вотс ап)").astype(int)

    return df


### Очистка данных

In [9]:
path_to_stopwords = "/home/kenny/avito_tech/data/raw/stopwords.txt"

In [49]:
def get_stopwords(path_to_stopwords):
    stopwords = []
    # при запуске через докер нужно будет поменять путь на app/data/raw/stopwords.txt
    with open(
        path_to_stopwords,
        encoding="utf-8",
    ) as f:
        for line in f:
            stopwords.append(line.strip("\n"))

    return stopwords

def clean_text(text: str) -> str:
    text = re.sub(r'[^0-9a-zA-Zа-яА-ЯёЁ\.,\(\)]+', ' ', text)
    text = re.sub(r'([^\w ])', r' \1', text)
    text = re.sub(r'([^ \w])', r'\1', text)
    text = re.sub(r' +', r' ', text)
    text = re.sub(r'^ ', r'', text)
    text = re.sub(r'[\W_]+', ' ', text)
    return text

@lru_cache(100_000_000)
def lemmatize(token, stemmer) -> str:
    return stemmer.parse(token)[0].normal_form

def remove_stopwords(text, path_to_stopwords):
    stop_words = get_stopwords(path_to_stopwords)
    return [word for word in text if word not in stop_words]

def preprocess_text(text, stemmer, path_to_stopwords):
    text = clean_text(text).split()
    text = remove_stopwords(text, path_to_stopwords)
    text = " ".join([lemmatize(token, stemmer) for token in text])
    return text

In [50]:
def preprocess_data(df, path_to_stopwords):
    df.description = df.description.fillna("")
    df.title = df.title.fillna("")
    df["text"] = df.title + " " +  df.description
    stemmer = MorphAnalyzer()
    df["text"] = df.text.progress_apply(lambda text: preprocess_text(text, stemmer, path_to_stopwords))
    df['filtered_text'] = df.text.progress_apply(
        lambda text: re.sub('[^A-Za-z0-9\.\@\ \-\_]', ' ', text)
    )
    df['filtered_text'] = df.filtered_text.progress_apply(
        lambda text: re.sub(' +', ' ', text)
    )
    df['filtered_numbers'] = df.text.progress_apply(
        lambda text: re.sub('[^0-9\+\(\)\-]', ' ', text)
    )
    df['filtered_numbers'] = df.filtered_numbers.progress_apply(
        lambda text: re.sub(' +', ' ', text)
    )
    return df

In [51]:
processed_train = preprocess_data(train, path_to_stopwords)

In [52]:
processed_train.sample(2)

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,text,filtered_text,filtered_numbers
72786,"Mercedes-Benz C-класс, 1009","Ошибок нет,масло не расходует вообще,работает ...",Автомобили,Транспорт,610000.0,Мурманская область,Мурманск,2019-06-09 18:45:46.583524,0.0,ошибка масло расходовать вообще работать отлич...,. . . . 6 .AMG . . . . . WDDGF84X39R077365 Me...,6 ( ) ( ) 84 39 077365 - - 1009
75171,Картофель,"Продам картофель крупный на еду, ведро 20л. за...",Продукты питания,Для дома и дачи,150.0,Кемеровская область,Новокузнецк,2019-06-10 09:21:33.934536,1.0,продать картофель крупный еда ведро 20л 260 ру...,20 . 260 . 20 . 79603722188,20 260 20 +79603722188


In [53]:
processed_train.iloc[1]

title                           Кожух рулевой колонки Даф хф 91 4509834
description           Кожух рулевой колонки DAF XF 94 (60066004)/\n ...
subcategory                                       Запчасти и аксессуары
category                                                      Транспорт
price                                                            2290.0
region                                                           Россия
city                                                             Москва
datetime_submitted                           2019-06-01 00:00:44.317933
is_bad                                                              0.0
text                  кожух рулевой колонка daf xf 94 60066004 артик...
filtered_text          DAF XF 94 60066004 2309862 . 262620 . . . . ....
filtered_numbers            94 (60066004) 2309862 262620 2 4 91 4509834
Name: 1, dtype: object

In [55]:
text = processed_train[["text", "filtered_text", "filtered_numbers"]].iloc[1].text
filtered_text = processed_train[["text", "filtered_text", "filtered_numbers"]].iloc[1].filtered_text
filtered_numbers = processed_train[["text", "filtered_text", "filtered_numbers"]].iloc[1].filtered_numbers

In [59]:
processed_train.iloc[1].description + processed_train.iloc[1].title

'Кожух рулевой колонки DAF XF 94 (60066004)/\n /\nартикул 2309862/\n /\nВ наличии на складе./\n /\nНомер детали 262620 сообщайте при бронировании./\n /\nУважаемые пользователи, в данный момент у нашей компании нет возможности оперативно отвечать на Ваши сообщения внутри Авито, в связи с чем данная функция отключена. По всем вопросам обращайтесь по телефону указанному в объявлении./\n/\nГрузовая разборка запчасти б/у./\n /\nВ наличии любые другие запчасти для грузовых Даф/\n /\nЕсть запчасти на все модели./\n/\nУ нас большой склад в России и Европе, поэтому звоните уточняйте по наличию./\n /\nПри необходимости отправляем запчасти в любые регионы, гарантия на запчасти 2 месяц!/\n /\nЗвоните, скажите что Вы с Авито и мы дадим скидку 4%, точно договоримся!Кожух рулевой колонки Даф хф 91 4509834'

In [56]:
text

'кожух рулевой колонка daf xf 94 60066004 артикул 2309862 в наличие склад номер деталь 262620 сообщать бронирование уважаемый пользователь данный момент наш компания возможность оперативно отвечать ваш сообщение внутри авить связь дать функция отключить по весь вопрос обращаться телефон указанный объявление грузовой разборка запчасть б в наличие любой другой запчасть грузовой даф есть запчасть модель у большой склад россия европа поэтому звонить уточнять наличие при необходимость отправлять запчасть любой регион гарантия запчасть 2 месяц звонить сказать вы авить дать скидка 4 точно договориться кожух рулевой колонка даф хф 91 4509834'

In [57]:
filtered_text

' DAF XF 94 60066004 2309862 . 262620 . . . . . . 2 4 91 4509834'

In [58]:
filtered_numbers

' 94 (60066004) 2309862 262620 2 4 91 4509834'